# Environment Setup

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F

In [2]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# Define transformations for the training set
transform = transforms.Compose([
    transforms.ToTensor(), # this will scaled from [0, 255] to [0, 1] when to tensor
    transforms.Normalize((0.5,), (0.5,)) # normalizes the tensor image with a mean of 0.5 and a standard deviation of 0.5. This will transform the pixel values to be approximately in the range [-1, 1].
])

# Load the MNIST dataset
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:02<00:00, 4905921.35it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 130085.16it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:01<00:00, 1263726.32it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 3871271.85it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



# Model

In [11]:
class ClassicCNN(nn.Module):
    def __init__(self):
        super(ClassicCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1) # 32 filters, filter size 3
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)
        self.dropout = nn.Dropout(p=0.25)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x))) # convolutional: [batch_size, channels, height, width]
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7) # Flatten the tensor for the fully connected layer
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [12]:
# Instantiate the model, define the loss function and the optimizer
model = ClassicCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training

In [13]:
num_epochs = 10

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:    # Print every 100 mini-batches
            print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 100:.3f}")
            running_loss = 0.0

print("Finished Training")

[1, 100] loss: 0.937
[1, 200] loss: 0.249
[1, 300] loss: 0.201
[1, 400] loss: 0.141
[1, 500] loss: 0.149
[1, 600] loss: 0.127
[1, 700] loss: 0.121
[1, 800] loss: 0.109
[1, 900] loss: 0.099
[1, 1000] loss: 0.106
[1, 1100] loss: 0.099
[1, 1200] loss: 0.081
[1, 1300] loss: 0.088
[1, 1400] loss: 0.089
[1, 1500] loss: 0.085
[1, 1600] loss: 0.062
[1, 1700] loss: 0.077
[1, 1800] loss: 0.077
[2, 100] loss: 0.059
[2, 200] loss: 0.063
[2, 300] loss: 0.070
[2, 400] loss: 0.062
[2, 500] loss: 0.061
[2, 600] loss: 0.056
[2, 700] loss: 0.061
[2, 800] loss: 0.051
[2, 900] loss: 0.065
[2, 1000] loss: 0.052
[2, 1100] loss: 0.060
[2, 1200] loss: 0.054
[2, 1300] loss: 0.061
[2, 1400] loss: 0.046
[2, 1500] loss: 0.069
[2, 1600] loss: 0.054
[2, 1700] loss: 0.050
[2, 1800] loss: 0.051
[3, 100] loss: 0.034
[3, 200] loss: 0.032
[3, 300] loss: 0.033
[3, 400] loss: 0.051
[3, 500] loss: 0.045
[3, 600] loss: 0.047
[3, 700] loss: 0.045
[3, 800] loss: 0.042
[3, 900] loss: 0.035
[3, 1000] loss: 0.044
[3, 1100] loss:

# Evaluation

In [14]:
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy of the network on the 10000 test images: {100 * correct / total}%")

Accuracy of the network on the 10000 test images: 98.88%
